# Parametric building genarator

#### Reference models:

## first model:

![img](model1.jpg)


## second model:
![img](model2.jpg)


#### Required libraries from previous workshops:
#### *workshop_3*: stairs
#### *workshop_7*: doors and windows
#### *workshop_8*: inside of the apartments
#### *workshop_9*: roof

#### The workshop_8 was changed in order to add the tool of windows and doors insertion.




In [ ]:
from pyplasm import *
import csv
import src.workshop_08 as create_walls
import src.workshop_07 as create_door_window
import src.workshop_03 as create_stairs
import src.workshop_09 as create_roof

# Builder 	
    
####  The main functions are: 
* *create_house*: is a function that returns a model representing the house. It takes as parameters: number of floors, .lines files, x,y,z representing the factor to scale.
* *create_windows*: takes as parameters the list of axis x and y of the window cells. This functions returns a model representing the windows of the house.
* *create_doors*: takes as parameters list of asix x and y of the doors cells, and bool matrix. 
* *create_floors*: takes as parameters a list of integer reppresenting the structure of the roof, aa angle to rotate the planes and a integer height of the roof.
* *create_build*: set a function that returns a palace with all parametric variable.
    
    	
  

In [ ]:
windowx = [2,3,2,3,2]
windowy = [2,3,2,3,2]
p = 15
t = .1
m = 1.09
occurrencwindowy = [[1, 1, 1, 1, 1],
			  [1, 0, 1, 0, 1],
			  [1, 1, 1, 1, 1],
			  [1, 0, 1, 0, 1],
			  [1, 1, 1, 1, 1]]

doorx = [.2, .2, .05, .2, .05, .2, .3, .2, .05, .2 ,.05, .2, .2]
doory = [.2, .2, .05, .2, .05, 1, .05, .2, .05, .2, .2]
occurencdoory = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
				[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
				[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1], 
				[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
				[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
				[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], 
				[1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1],
				[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
				[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
				[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
				[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

external_walls = STRUCT(create_walls.create_walls('fmme1.lines'))
x = (p)/SIZE([1])(external_walls)[0]
y = (p+t)/SIZE([2])(external_walls)[0]
z = x

external_walls2 = create_walls.create_walls('smme1.lines')
externalWalls2 = STRUCT(external_walls2)
x2 = p/SIZE([1])(externalWalls2)[0]
y2 = p+t/SIZE([2])(externalWalls2)[0]
z2 = x2



def get_points_from_csv(path, filename):

    path = "".join([path, filename])
    csv_file = open(path, "r")
    csv_points = []

    for line in csv_file:
        line_split = line.split(",")

        if line_split[1] == "floors" or line_split[1] == "thickness":
            lat = line_split[1]
            lon = int(line_split[2])

        else:
            lat = float(line_split[1])
            lon = float(line_split[2])

        csv_points.append([lat, lon])

    csv_file.close()
    return csv_points

class Building:

    def __init__(self, perimetral_points, floors):
        
        self.points = perimetral_points
        self.floors = floors
        self.building = self.make_building(perimetral_points, floors)
        
    def make_building(self, perimetral_points, floors):
        
        exterior_texture = get_texture("building")
        
        wall_list = []
        for i in range(len(perimetral_points)-1):
            
            wall_list_per_floor = []
            for k in range(floors):
                vect = [perimetral_points[i], perimetral_points[i+1]]
                distance = distanza_euclidea(perimetral_points[i], 
                                             perimetral_points[i+1])
                wall = TEXTURE(exterior_texture)(
                    CUBOID([distance, FLOOR_SIZE, WALL_THICKNESS]))

                if (distance >= WINDOW_WIDTH):
                
                    window_shift_vector = T([1,2,3])([float(distance)/2,
                                                      float(FLOOR_SIZE)/2,
                                                      0])
                    window = window_shift_vector(
                        create_window(WINDOW_WIDTH,WINDOW_HEIGHT))
                   
                    dummy_window = window_shift_vector(
                        T([1,2])([-WINDOW_WIDTH/2,-WINDOW_HEIGHT/2])(
                            CUBOID([WINDOW_WIDTH,WINDOW_HEIGHT,WALL_THICKNESS])))                      
                    wall = TEXTURE([exterior_texture,
                                    True, True,
                                    0,0,0,2,2,1,1])(DIFFERENCE([wall, dummy_window]))
                    wall = STRUCT([wall, window])
                wall = R([2,3])(PI/2)(wall)
               
                wall = R([1,2])(alignament_angle(vect))(wall)
    

                wall = T(3)(k*FLOOR_SIZE)(wall)
                wall = T([1,2])([perimetral_points[i][0], perimetral_points[i][1]])(wall)
                wall_list_per_floor.append(wall)
            
            wall_list.append(STRUCT(wall_list_per_floor))
            
        
            
        roof = create_roof(perimetral_points, floors, exterior_texture)
        return STRUCT([roof, STRUCT(wall_list)])

    
    #----------------------------------------------------------------------
    def render(self):

        return self.building
    
    #----------------------------------------------------------------------
    def render_with_altitude(self, terrain_mesh):

        
        perimetral_points_avg = average(array(self.points), axis = 0)
        lat, lon = cartesian2latlon(perimetral_points_avg)
        return T(3)(terrain_mesh.get_altitude(lat, lon))(self.building)


def create_r(perimetral_points, floors, e_texture):


    if floors <=2 and len(perimetral_points) == 5:
        distances_list = []
        for i in range(len(perimetral_points)-1):
            distance = distanza_euclidea(perimetral_points[i], 
                                             perimetral_points[i+1])
            distances_list.append(distance)
        
        distances_list.sort(reverse = True)
        
        roof_list = []
        pt_list = []
        for d in distances_list[0:2]:
            for i in range(len(perimetral_points)-1):
                du = distanza_euclidea(perimetral_points[i], perimetral_points[i+1])
                if du == d:
                    p1 = [perimetral_points[i][0], perimetral_points[i][1], 0]
                    p2 = [perimetral_points[i+1][0], perimetral_points[i+1][1], 0]
                    pm_x = (perimetral_points[i][0] + perimetral_points[i+1][0])/2
                    pm_y = (perimetral_points[i][1] + perimetral_points[i+1][1])/2
                    punto_mediano = [pm_x, pm_y, ROOF_HEIGHT]
                    pt_list.append([p1, punto_mediano, p2, p1])
                    roof = COLOR(RED)(POLYLINE([p1, punto_mediano, p2, p1]))
                    roof_list.append(roof)
                    
        roof_holder = [TEXTURE([e_texture, True, True,
                                0,0,90,2,2,0,0])(JOIN(p)) for p in roof_list]
        roof_holder = T(3)(floors*FLOOR_SIZE)(STRUCT(roof_holder))

        coperchio1 = POLYLINE([pt_list[1][0], pt_list[1][1],
                               pt_list[0][1], pt_list[0][2],
                               pt_list[1][0]])
        coperchio2 = POLYLINE([pt_list[1][1], pt_list[1][2],
                               pt_list[0][0], pt_list[0][1],
                               pt_list[1][1]])
        
        roof_texture = get_texture("high_roof")
        
        coperchio1 = OFFSET([0,0,2])(JOIN(coperchio1))
        coperchio2 = OFFSET([0,0,2])(JOIN(coperchio2))
        coperchio1 = TEXTURE(roof_texture)(T(3)(floors*FLOOR_SIZE)(coperchio1))
        coperchio2 = TEXTURE(roof_texture)(T(3)(floors*FLOOR_SIZE)(coperchio2))

        return STRUCT([roof_holder, coperchio1, coperchio2])
                    
            
   
    if floors >=3 or len(perimetral_points) > 5:
        roof = []
        for i in range(len(perimetral_points)-1):
            roof.append(POLYLINE([perimetral_points[i],
                                  perimetral_points[i+1]]))
                
        roof = EXTRUDE([3, SOLIDIFY3(STRUCT(roof)), WALL_THICKNESS])    
        roof = TEXTURE(get_texture("roof"))(T(3)(FLOOR_SIZE*floors)(roof))
        return roof
    
    
    
class Eco(Building):
    def __init__(self, perimetral_points, floors = 1):
        
        self.points = perimetral_points
        self.floors = floors
        self.building = self.make_ecomostro(perimetral_points, floors)
    
    
    def make_eco(self, perimetral_points, floors = 1):

        FLOOR_SIZE = 250
        WALL_THICKNESS = 4
        WINDOW_WIDTH = 50
        WINDOW_HEIGHT = 100
        exterior_texture = "".join([TEXTURES_DIR, "cemento.png"])
        
        wall_list = []
        for i in range(len(perimetral_points)-1):
            
            wall_list_per_floor = []
            for k in range(floors):
                vect = [perimetral_points[i], perimetral_points[i+1]]
                distance = distanza_euclidea(perimetral_points[i], 
                                             perimetral_points[i+1])
                wall = TEXTURE(exterior_texture)(
                    CUBOID([distance, FLOOR_SIZE, WALL_THICKNESS]))
    
                if (distance >= WINDOW_WIDTH):
                
                    window_shift_vector = T([1,2,3])([float(distance)/2,
                                                      float(FLOOR_SIZE)/2,
                                                      0])
                    window = window_shift_vector(
                        create_window(WINDOW_WIDTH,WINDOW_HEIGHT))
                    dummy_window = window_shift_vector(
                        T([1,2])([-WINDOW_WIDTH/2,-WINDOW_HEIGHT/2])(
                            CUBOID([WINDOW_WIDTH,WINDOW_HEIGHT,WALL_THICKNESS])))                      
                    wall = TEXTURE([exterior_texture,
                                    True, True,
                                    0,0,0,2,2,1,1])(DIFFERENCE([wall, dummy_window]))
                wall = R([2,3])(PI/2)(wall)

                wall = R([1,2])(alignament_angle(vect))(wall)
    
                wall = T(3)(k*FLOOR_SIZE)(wall)
                wall = T([1,2])([perimetral_points[i][0], perimetral_points[i][1]])(wall)
                wall_list_per_floor.append(wall)
            
            wall_list.append(STRUCT(wall_list_per_floor))
            
        
            
        roof = []
        for i in range(len(perimetral_points)-1):
            roof.append(POLYLINE([perimetral_points[i],
                                  perimetral_points[i+1]]))
                
        roof = EXTRUDE([3, SOLIDIFY3(STRUCT(roof)), WALL_THICKNESS])    
        roof = TEXTURE(exterior_texture)(T(3)(FLOOR_SIZE*floors)(roof))
        return STRUCT([roof, STRUCT(wall_list)])


def create_w(width, height):
    
    offset_z = 50.0/(1000/height)
    sf_y = 0.05
    sf_x = sf_y * (float(height)/float(width))
    
    scocca_esterna1 = POLYLINE([[float(width)/2,0],
                               [0,0],
                               [0, height],
                               [float(width)/2, height]])
    scocca_esterna2 = STRUCT([T([1, 2])([(float(width)/2*sf_x), sf_y*height/2]),
                              S([1,2])([1 - sf_x, 1 - sf_y]),
                              scocca_esterna1])
    
    scocca_esterna = STRUCT([scocca_esterna1, scocca_esterna2])
    scocca_esterna_mirrored = T(1)(width)(S(1)(-1)(scocca_esterna))
    scocca_esterna = STRUCT([scocca_esterna, scocca_esterna_mirrored])
    scocca_esterna = EXTRUDE([3, SOLIDIFY2(scocca_esterna), offset_z])
    
    scocca_interna1 = STRUCT([T([1,2])([(float(width)/2*sf_x), sf_y*height/2]),
                             S([1,2,3])([0.55-sf_x, 1 - sf_y, 0.5]),
                             scocca_esterna])

    scocca_interna1 = TEXTURE(get_texture("window"))(scocca_interna1)
    vetro = STRUCT([T([1,2,3])([0.5*(float(width)/2*sf_x), sf_y*height/2]),
                             S([1,2,3])([1 - sf_x, 1 - sf_y, 0.5]),
                             scocca_interna1])
    vetro = MATERIAL([1,1,1,0.1,  0,0,0.8,0.5,  1,1,1,0.1,  1,1,1,0.1,  100])(JOIN(vetro))
    scocca_e_vetro1 = STRUCT([scocca_interna1, vetro])
    scocca_interna2 = T(1)(0.45*float(width))(scocca_e_vetro1)

    finestra = STRUCT([scocca_esterna, scocca_e_vetro1, scocca_interna2])

    return STRUCT([T(1)(-float(width)/2),T(2)(float(height)/2),R([2,3])(PI),finestra])


def make_build():

    files = os.listdir(BUILDINGS_DIR)
    
    buildings_list = [get_points_from_csv(BUILDINGS_DIR, f) for f in files
                       if not hidden_file(f)]
    floors = [l.pop()[1] for l in buildings_list]
    buildings_list = [latlon2cartesian_list(b) for b in buildings_list]
    buildings = [Building(buildings_list[i],
                          floors[i]) for i in range(len(buildings_list))]
    return buildings

	
    

def create_house(floornum, bases, x, y, z):
	def create_windows(windowx, windowy, occurrencwindowy):
		def create_doors(doorx, doory, occurencdoory):	
			def create_floors(v, angle, h):		
				floors = []
				#building roof model
				with open(v) as file:
					reader = csv.reader(file, delimiter=",")
					new_v = []
					for row in reader:
						new_v.append([float(row[0]), float(row[1])])
					roof = T([3])([floornum*3/z])(create_roof.buildRoof(new_v, angle, h))
					roof = S([1,2,3])([x*m, y*m, z])(roof)
					roof = T([1,2])([-SIZE([1])(roof)[0]*(t-.05), -SIZE([2])(roof)[0]*(t-.05)])(roof)
				for i in range(floornum):
					floor_lines = [bases + '_me_'+str(i+1)+'.lines', bases + '_mi_'+str(i+1)+'.lines', bases + '_p_'+str(i+1)+'.lines', bases + '_f_'+str(i+1)+'.lines', bases + '_s_'+str(i)+'.lines', bases + '_s_'+str(i+1)+'.lines']
					floor = create_walls.ggpl_building_house(floor_lines, 
						create_door_window.window_creator(windowx,windowy,occurrencwindowy), 
						create_door_window.door_creator(doory, doorx, occurencdoory), 
						create_stairs, i, floornum-1)
					floors.append(floor)			
				floors = STRUCT(floors)
				return STRUCT([floors, roof])
			return create_floors
		return create_doors
	return create_windows



# Tests an results


### To ensure the dynamism of the function, tests have been conducted on the base of two different kinds of parametric data.



#### External first house
![img](house2.png)

![img](house2.1.png)

#### Room first house
![img](house2.2.png)


#### Second house(external):
![img](house1.png)


#### Roof second house
![img](house1.1.png)


#### The staircase inside the house:
![img](house1.2.png)


#### Room inside second house
![img](house1.3.png)


#### A palace
![img](palace.png)




